In [1]:
import os 
os.environ["CUDA_VISIBLE_DEVICES"]="1"
from imports import *
from data_generator import *
from models import *
import time
from utils import limit_mem

Using TensorFlow backend.


In [3]:
DATADIR = 'Preprocessed_Data/7_Years_Spaced/'

train_gen = DataGenerator(
    data_dir=DATADIR, 
    feature_fn='full_physics_essentials_train_month01_shuffle_features.nc',
    target_fn='full_physics_essentials_train_month01_shuffle_targets.nc',
    batch_size=512,
    norm_fn='full_physics_essentials_train_month01_norm.nc',
    fsub='feature_means', 
    fdiv='feature_stds', 
    tmult='target_conv',
    shuffle=True,
)

valid_gen = DataGenerator(
    data_dir=DATADIR, 
    feature_fn='full_physics_essentials_valid_month02_features.nc',
    target_fn='full_physics_essentials_valid_month02_targets.nc',
    batch_size=512,
    norm_fn='full_physics_essentials_train_month01_norm.nc',  # SAME NORMALIZATION FILE!
    fsub='feature_means', 
    fdiv='feature_stds', 
    tmult='target_conv',
    shuffle=False,
)

Generator will have 378224640 samples in 738720 batches
Features have shape 64; targets have shape 65
Generator will have 47278080 samples in 92340 batches
Features have shape 64; targets have shape 65


In [6]:
n = 64
x = tf.placeholder(tf.float32, shape=(n, None))
y = tf.placeholder(tf.float32, shape=(1, None))

# Define trainable variables
A = tf.get_variable("A", shape=(1, n))
b = tf.get_variable("b", shape=())

# Define model output
y_predicted = tf.matmul(A, x) + b

# Define the loss function
L = tf.reduce_sum((y_predicted - y)**2)

# Define optimizer object
optimizer = tf.train.AdamOptimizer(learning_rate=0.1).minimize(L)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [12]:
print(train_gen.return_generator(True,True))
print(train_gen.n_batches)

738720


In [14]:
tf.keras.experimental.LinearModel()

In [13]:
session = tf.Session()
session.run(tf.global_variables_initializer())

# Main optimization loop
for t in range(2000):
    _, current_loss, current_A, current_b = session.run([optimizer, L, A, b], feed_dict={
        x: train_gen.return_generator(False,False),
        y: train_gen.return_generator(False,False)
    })
    print("t = %g, loss = %g, A = %s, b = %g" % (t, current_loss, str(current_A), current_b))


TypeError: float() argument must be a string or a number, not 'threadsafe_iter'